#### AAI 521 - <b><font color='Red'>Final Project</font></b>
- This module author: <b><font color='Red'>PRAKASH PERIMBETI</font></b>
- TEAM: Christopher Teli (Lead), Olympia Saha and Prakash Perimbeti
- Date: December 12, 2022
- Professor: Dr. Saeed Sardari

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
from IPython.display import Image, display, Markdown
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from time import perf_counter
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.layers as tfl
from keras.optimizers import SGD   
from  sklearn.preprocessing import LabelEncoder 

2022-12-08 20:38:55.472470: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-08 20:38:55.472722: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-08 20:38:55.472756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [34]:
#Get image file names and its classifiers
trainimage_dir = Path('./train')
trainlabels = []
# Get filepaths and labels
#filepaths =  os.listdir(image_dir)
Tfilepaths = list(trainimage_dir.glob(r'**/*.jpg'))
count = 0 

for filename in filepaths:
    lbl = str(filename)[6:7]
    trainlabels.append(lbl)
    count+=1

print(len(Tfilepaths), len(trainlabels))
print(Tfilepaths[4000], trainlabels[4000])


# total 2515 images
filepaths = pd.Series(Tfilepaths, name='Filepath').astype(str)
labels = pd.Series(trainlabels, name='Label')

# Concatenate filepaths and labels
T_IMAGE_DF = pd.concat([filepaths, labels], axis=1)

# Shuffle the DataFrame and reset index
T_IMAGE_DF = T_IMAGE_DF.sample(frac=1).reset_index(drop = True)

#le_Label_ACTIVE = LabelEncoder()
#IMAGE_DF['Label'] = le_Label_ACTIVE.fit_transform(IMAGE_DF['Label'])

#IMAGE_DF['Label'] = pd.Categorical(IMAGE_DF.Label)
print(T_IMAGE_DF.shape)
# Show the result
T_IMAGE_DF.head(5)
# Separate in train and test data
T_TRAIN_DF, T_TEST_DF = train_test_split(T_IMAGE_DF, train_size=0.9, shuffle=True, random_state=1)
print(T_IMAGE_DF.shape, T_TRAIN_DF.shape, T_TEST_DF.shape)
# Create the generators
T_train_generator,T_test_generator,T_TRAIN_IMAGES,T_VAL_IMAGES,T_TEST_IMAGES=create_gen(T_TRAIN_DF, T_TEST_DF)
print('\n')

4972 4972
train/N/088.jpg N
(4972, 2)
(4972, 2) (4474, 2) (498, 2)
Found 4027 validated image filenames belonging to 24 classes.
Found 447 validated image filenames belonging to 24 classes.
Found 498 validated image filenames belonging to 24 classes.




In [ ]:
#Get image file names and its classifiers
image_dir = Path('./imagedata')
labels = []
# Get filepaths and labels
#filepaths =  os.listdir(image_dir)
filepaths = list(image_dir.glob(r'**/*.png'))
count = 0 
for filename in filepaths:
    labels.append(str(filename[6]))
    filepaths[count] = './imagedata' + '/' + filename
    count+=1
    

In [6]:
from  sklearn.preprocessing import LabelEncoder 
# total 2515 images
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
IMAGE_DF = pd.concat([filepaths, labels], axis=1)

# Shuffle the DataFrame and reset index
IMAGE_DF = IMAGE_DF.sample(frac=1).reset_index(drop = True)

#le_Label_ACTIVE = LabelEncoder()
#IMAGE_DF['Label'] = le_Label_ACTIVE.fit_transform(IMAGE_DF['Label'])

#IMAGE_DF['Label'] = pd.Categorical(IMAGE_DF.Label)
print(IMAGE_DF.shape)
# Show the result
IMAGE_DF.head(5)

(2515, 2)


,Filepath,Label
0,./imagedata/hand1_c_dif_seg_5_cropped.png,c
1,./imagedata/hand3_b_dif_seg_2_cropped.png,b
2,./imagedata/hand1_u_top_seg_4_cropped.png,u
3,./imagedata/hand2_a_dif_seg_1_cropped.png,a
4,./imagedata/hand2_f_left_seg_3_cropped.png,f


In [7]:
print(len(filepaths), len(labels), labels[4])

2515 2515 r


In [8]:
# Separate in train and test data
TRAIN_DF, TEST_DF = train_test_split(IMAGE_DF, train_size=0.9, shuffle=True, random_state=1)
print(IMAGE_DF.shape, IMAGE_DF.shape, IMAGE_DF.shape)

(2515, 2) (2515, 2) (2515, 2)


In [9]:
#8586586550
#
# This loads images using Image Data Generator
def create_gen(train_df, test_df):
    # Load the Images with a generator and Data Augmentation
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.1
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        seed=0,
        subset='training',
        rotation_range=30, # Uncomment to use data augmentation
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        seed=0,
        subset='validation',
        rotation_range=30, # Uncomment to use data augmentation
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )

    return train_generator,test_generator,train_images,val_images,test_images

In [10]:
# Create the generators
train_generator,test_generator,TRAIN_IMAGES,VAL_IMAGES,TEST_IMAGES=create_gen(TRAIN_DF, TEST_DF)

print('\n')

Found 2037 validated image filenames belonging to 36 classes.
Found 226 validated image filenames belonging to 36 classes.
Found 252 validated image filenames belonging to 36 classes.




In [ ]:
saved = buildTransferModels(TRAIN_IMAGES, VAL_IMAGES)
print( "A total of :", saved," models from this build")

processing: InceptionResNetV2
fitting the model: InceptionResNetV2
Epoch 1/10
64/64 [==============================] - 262s 4s/step - loss: 3.6287 - accuracy: 0.0196 - val_loss: 3.6198 - val_accuracy: 0.0310
Epoch 2/10
64/64 [==============================] - 218s 3s/step - loss: 3.5465 - accuracy: 0.0422 - val_loss: 3.5625 - val_accuracy: 0.0310
Epoch 3/10
64/64 [==============================] - 217s 3s/step - loss: 3.5042 - accuracy: 0.0555 - val_loss: 3.5255 - val_accuracy: 0.0487
Epoch 4/10
64/64 [==============================] - 218s 3s/step - loss: 3.4727 - accuracy: 0.0717 - val_loss: 3.4980 - val_accuracy: 0.0929
Epoch 5/10
64/64 [==============================] - 217s 3s/step - loss: 3.4448 - accuracy: 0.1011 - val_loss: 3.4719 - val_accuracy: 0.1283
Epoch 6/10
64/64 [==============================] - 218s 3s/step - loss: 3.4183 - accuracy: 0.1306 - val_loss: 3.4472 - val_accuracy: 0.1372
Epoch 7/10
64/64 [==============================] - 218s 3s/step - loss: 3.3909 - accur

In [10]:
sgd = SGD(learning_rate=0.001)
def get_model(model):
# Load the pretained model
    kwargs =    {'input_shape':(224, 224, 3),
                'include_top':False,
                'weights':'imagenet',
                'pooling':'avg'}
    
    pretrained_model = model(**kwargs)
    pretrained_model.trainable = False
    
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(36, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=sgd,#'adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [38]:
# Dictionary with the models
models = {
    "InceptionResNetV2": {"model":tf.keras.applications.InceptionResNetV2, "perf":0},
    "ResNet101": {"model":tf.keras.applications.ResNet101, "perf":0},
    "InceptionV3": {"model":tf.keras.applications.InceptionV3, "perf":0},
    "DenseNet121": {"model":tf.keras.applications.DenseNet121, "perf":0},
    "MobileNet": {"model":tf.keras.applications.MobileNet, "perf":0},
    "VGG16": {"model":tf.keras.applications.VGG16, "perf":0},
}

def buildTransferModels(TRAIN_IMAGES, VAL_IMAGES):

  # Fit the models
  takeNext = False
  savedModels = 0
  for name, model in models.items():
    print("processing:", name)
    # Get the model
    m = get_model(model['model'])
    models[name]['model'] = m
    start = perf_counter()
    print('fitting the model:', name)
    # Fit the model
    try:
        history = m.fit(TRAIN_IMAGES,validation_data=VAL_IMAGES,epochs=10,verbose=1) 
    except Exception:
        print(Exception)
        print("Failed with:",name," Transfer learnning")
        takeNext = True
    if(takeNext is False):
        # Sav the duration, the train_accuracy and the val_accuracy
        duration = perf_counter() - start
        duration = round(duration,2)
        models[name]['perf'] = duration
        print(f"{name:20} trained in {duration} sec")
    
        val_acc = history.history['val_accuracy']
        models[name]['val_acc'] = [round(v,4) for v in val_acc]
    
        train_acc = history.history['accuracy']
        models[name]['train_accuracy'] = [round(v,4) for v in train_acc]
        m.save(name+".h5")
        savedModels +=1
  return savedModels


In [11]:
#Get image file names and its classifiers
image_dir = Path('./imagedata')
labels = []
# Get filepaths and labels
filepaths =  os.listdir(image_dir)
#filepaths = list(image_dir.glob(r'**/*.png'))
count = 0 

for filename in filepaths:
    lbl = str(filename)[6:7]
    labels.append(lbl)
    filepaths[count] = './imagedata'+'/'+filename
    count+=1

print(len(filepaths), len(labels))
print(filepaths[400], labels[400])


# total 2515 images
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
IMAGE_DF = pd.concat([filepaths, labels], axis=1)

# Shuffle the DataFrame and reset index
IMAGE_DF = IMAGE_DF.sample(frac=1).reset_index(drop = True)

#le_Label_ACTIVE = LabelEncoder()
#IMAGE_DF['Label'] = le_Label_ACTIVE.fit_transform(IMAGE_DF['Label'])

#IMAGE_DF['Label'] = pd.Categorical(IMAGE_DF.Label)
print(IMAGE_DF.shape)
# Show the result
IMAGE_DF.head(5)
# Separate in train and test data
TRAIN_DF, TEST_DF = train_test_split(IMAGE_DF, train_size=0.9, shuffle=True, random_state=1)
print(IMAGE_DF.shape, TRAIN_DF.shape, TEST_DF.shape)
# Create the generators
train_generator,test_generator,TRAIN_IMAGES,VAL_IMAGES,TEST_IMAGES=create_gen(TRAIN_DF, TEST_DF)
print('\n')

2515 2515
./imagedata/hand2_q_right_seg_3_cropped.png q
(2515, 2)
(2515, 2) (2263, 2) (252, 2)
Found 2037 validated image filenames belonging to 36 classes.
Found 226 validated image filenames belonging to 36 classes.
Found 252 validated image filenames belonging to 36 classes.




In [12]:
# Dictionary with the models
models = {
    "InceptionResNetV2": {"model":tf.keras.applications.InceptionResNetV2, "perf":0},
    "ResNet101": {"model":tf.keras.applications.ResNet101, "perf":0},
    "InceptionV3": {"model":tf.keras.applications.InceptionV3, "perf":0},
    "DenseNet121": {"model":tf.keras.applications.DenseNet121, "perf":0},
    "MobileNet": {"model":tf.keras.applications.MobileNet, "perf":0},
    "VGG16": {"model":tf.keras.applications.VGG16, "perf":0},
}

def buildTransferModels(TRAIN_IMAGES, VAL_IMAGES, name):

    model = models.get(name)
    print("processing:", name)
    # Get the model
    m = get_model(model['model'])
    models[name]['model'] = m
    start = perf_counter()
    print('fitting the model:', name)
    history = m.fit(TRAIN_IMAGES,
                    validation_data=VAL_IMAGES,
                    epochs=10,
                    verbose=1,
                   batch_size = 32)

    # Sav the duration, the train_accuracy and the val_accuracy
    duration = perf_counter() - start
    duration = round(duration,2)
    models[name]['perf'] = duration
    print(f"{name:20} trained in {duration} sec")
    
    val_acc = history.history['val_accuracy']
    models[name]['val_acc'] = [round(v,4) for v in val_acc]
    
    train_acc = history.history['accuracy']
    models[name]['train_accuracy'] = [round(v,4) for v in train_acc]
    m.save(name+"36.h5")


In [29]:
buildTransferModels(TRAIN_IMAGES,VAL_IMAGES, "InceptionResNetV2")

processing: InceptionResNetV2
fitting the model: InceptionResNetV2
Epoch 1/10
64/64 [==============================] - 261s 4s/step - loss: 3.6702 - accuracy: 0.0515 - val_loss: 3.5829 - val_accuracy: 0.0796
Epoch 2/10
64/64 [==============================] - 220s 3s/step - loss: 3.5472 - accuracy: 0.0736 - val_loss: 3.5001 - val_accuracy: 0.0664
Epoch 3/10
64/64 [==============================] - 221s 3s/step - loss: 3.4818 - accuracy: 0.0781 - val_loss: 3.4555 - val_accuracy: 0.1018
Epoch 4/10
64/64 [==============================] - 220s 3s/step - loss: 3.4407 - accuracy: 0.0967 - val_loss: 3.4246 - val_accuracy: 0.1062
Epoch 5/10
64/64 [==============================] - 220s 3s/step - loss: 3.4056 - accuracy: 0.1159 - val_loss: 3.3924 - val_accuracy: 0.1239
Epoch 6/10
64/64 [==============================] - 220s 3s/step - loss: 3.3749 - accuracy: 0.1232 - val_loss: 3.3651 - val_accuracy: 0.1195
Epoch 7/10
64/64 [==============================] - 219s 3s/step - loss: 3.3460 - accur

In [7]:
buildTransferModels(TRAIN_IMAGES,VAL_IMAGES, "ResNet101")

processing: ResNet101


2022-12-08 20:41:04.739573: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-12-08 20:41:04.739666: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


fitting the model: ResNet101
Epoch 1/10
64/64 [==============================] - 309s 5s/step - loss: 3.6332 - accuracy: 0.0329 - val_loss: 3.6008 - val_accuracy: 0.0265
Epoch 2/10
64/64 [==============================] - 287s 4s/step - loss: 3.5709 - accuracy: 0.0432 - val_loss: 3.5778 - val_accuracy: 0.0531
Epoch 3/10
64/64 [==============================] - 284s 4s/step - loss: 3.5550 - accuracy: 0.0560 - val_loss: 3.5690 - val_accuracy: 0.0398
Epoch 4/10
64/64 [==============================] - 299s 5s/step - loss: 3.5452 - accuracy: 0.0599 - val_loss: 3.5536 - val_accuracy: 0.0619
Epoch 5/10
64/64 [==============================] - 282s 4s/step - loss: 3.5344 - accuracy: 0.0717 - val_loss: 3.5485 - val_accuracy: 0.0531
Epoch 6/10
64/64 [==============================] - 280s 4s/step - loss: 3.5258 - accuracy: 0.0707 - val_loss: 3.5365 - val_accuracy: 0.0619
Epoch 7/10
64/64 [==============================] - 281s 4s/step - loss: 3.5175 - accuracy: 0.0707 - val_loss: 3.5312 - val_a

In [ ]:
buildTransferModels(TRAIN_IMAGES,VAL_IMAGES, "InceptionV3")

processing: InceptionV3
fitting the model: InceptionV3
Epoch 1/10
64/64 [==============================] - 110s 2s/step - loss: 3.6607 - accuracy: 0.0368 - val_loss: 3.6218 - val_accuracy: 0.0177
Epoch 2/10
64/64 [==============================] - 93s 1s/step - loss: 3.5344 - accuracy: 0.0515 - val_loss: 3.5478 - val_accuracy: 0.0398
Epoch 3/10
64/64 [==============================] - 93s 1s/step - loss: 3.4589 - accuracy: 0.0697 - val_loss: 3.4916 - val_accuracy: 0.0664
Epoch 4/10
64/64 [==============================] - 93s 1s/step - loss: 3.3942 - accuracy: 0.0913 - val_loss: 3.4290 - val_accuracy: 0.1150
Epoch 5/10
64/64 [==============================] - 92s 1s/step - loss: 3.3314 - accuracy: 0.1217 - val_loss: 3.3771 - val_accuracy: 0.1239
Epoch 6/10
64/64 [==============================] - 92s 1s/step - loss: 3.2698 - accuracy: 0.1492 - val_loss: 3.3159 - val_accuracy: 0.1549
Epoch 7/10
55/64 [========================>.....] - ETA: 11s - loss: 3.2109 - accuracy: 0.1773

In [13]:
buildTransferModels(TRAIN_IMAGES,VAL_IMAGES, "DenseNet121")

processing: DenseNet121
fitting the model: DenseNet121
Epoch 1/10
64/64 [==============================] - 178s 2s/step - loss: 3.6562 - accuracy: 0.0422 - val_loss: 3.5955 - val_accuracy: 0.0354
Epoch 2/10
64/64 [==============================] - 153s 2s/step - loss: 3.5466 - accuracy: 0.0457 - val_loss: 3.5342 - val_accuracy: 0.0398
Epoch 3/10
64/64 [==============================] - 153s 2s/step - loss: 3.4956 - accuracy: 0.0599 - val_loss: 3.4945 - val_accuracy: 0.0619
Epoch 4/10
64/64 [==============================] - 151s 2s/step - loss: 3.4568 - accuracy: 0.0879 - val_loss: 3.4615 - val_accuracy: 0.0708
Epoch 5/10
64/64 [==============================] - 151s 2s/step - loss: 3.4223 - accuracy: 0.1046 - val_loss: 3.4276 - val_accuracy: 0.0841
Epoch 6/10
64/64 [==============================] - 148s 2s/step - loss: 3.3889 - accuracy: 0.1183 - val_loss: 3.3980 - val_accuracy: 0.0973
Epoch 7/10
64/64 [==============================] - 146s 2s/step - loss: 3.3570 - accuracy: 0.1306 

In [14]:
buildTransferModels(TRAIN_IMAGES,VAL_IMAGES, "MobileNet")

processing: MobileNet
fitting the model: MobileNet
Epoch 1/10
64/64 [==============================] - 47s 649ms/step - loss: 3.6730 - accuracy: 0.0344 - val_loss: 3.5447 - val_accuracy: 0.0487
Epoch 2/10
64/64 [==============================] - 39s 606ms/step - loss: 3.5695 - accuracy: 0.0550 - val_loss: 3.4907 - val_accuracy: 0.0885
Epoch 3/10
64/64 [==============================] - 39s 604ms/step - loss: 3.5218 - accuracy: 0.0746 - val_loss: 3.4571 - val_accuracy: 0.1062
Epoch 4/10
64/64 [==============================] - 39s 600ms/step - loss: 3.4863 - accuracy: 0.0908 - val_loss: 3.4268 - val_accuracy: 0.1062
Epoch 5/10
64/64 [==============================] - 39s 599ms/step - loss: 3.4542 - accuracy: 0.1105 - val_loss: 3.3978 - val_accuracy: 0.1504
Epoch 6/10
64/64 [==============================] - 38s 594ms/step - loss: 3.4222 - accuracy: 0.1325 - val_loss: 3.3678 - val_accuracy: 0.1460
Epoch 7/10
64/64 [==============================] - 39s 608ms/step - loss: 3.3901 - accurac

In [15]:
buildTransferModels(TRAIN_IMAGES,VAL_IMAGES, "VGG16")

processing: VGG16
fitting the model: VGG16
Epoch 1/10
64/64 [==============================] - 455s 7s/step - loss: 3.6359 - accuracy: 0.0270 - val_loss: 3.6567 - val_accuracy: 0.0221
Epoch 2/10
64/64 [==============================] - 451s 7s/step - loss: 3.6127 - accuracy: 0.0270 - val_loss: 3.6355 - val_accuracy: 0.0221
Epoch 3/10
64/64 [==============================] - 451s 7s/step - loss: 3.5988 - accuracy: 0.0285 - val_loss: 3.6201 - val_accuracy: 0.0221
Epoch 4/10
64/64 [==============================] - 452s 7s/step - loss: 3.5889 - accuracy: 0.0383 - val_loss: 3.6082 - val_accuracy: 0.0265
Epoch 5/10
64/64 [==============================] - 456s 7s/step - loss: 3.5816 - accuracy: 0.0432 - val_loss: 3.6005 - val_accuracy: 0.0398
Epoch 6/10
64/64 [==============================] - 454s 7s/step - loss: 3.5767 - accuracy: 0.0481 - val_loss: 3.5955 - val_accuracy: 0.0398
Epoch 7/10
64/64 [==============================] - 452s 7s/step - loss: 3.5730 - accuracy: 0.0501 - val_loss: 

In [34]:
m =  models["InceptionResNetV2"]['model']
ypredictionRN2 = m.predict(TEST_IMAGES)

8/8 [==============================] - 34s 3s/step


In [39]:
print(len(ypredictionRN2), ypredictionRN2[52])
def getSinglePredictedMaxColumn (prediction):
    num_rows, num_cols = prediction.shape
    print(num_rows, num_cols)
    new_list = []
    for i in range(num_rows):
        column_list = list(prediction[i])
        max_index = column_list.index(max(column_list))
        new_list.append(max_index)

    return(new_list)
newpred = getSinglePredictedMaxColumn(ypredictionRN2)
print(newpred[1:5])

252 [0.01390894 0.03801033 0.03917402 0.04834156 0.02821347 0.03703311
 0.04156258 0.0402364  0.02876344 0.01913362 0.02182228 0.02284929
 0.02479278 0.0215046  0.03413737 0.04300565 0.02044649 0.0157331
 0.03517412 0.03720588 0.02197585 0.02556166 0.03123729 0.01775581
 0.02419085 0.01927164 0.01762393 0.02654485 0.02252811 0.02427919
 0.02323967 0.01771199 0.04088639 0.01941895 0.02193448 0.03479033]
252 36
[32, 5, 5, 27]
